In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import os

# These are the column names:
"""
Unique: 
tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,round,minutes

prefixed with winner/loser:
winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points

prefixed with w/l:
w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced
"""
BASE_DIR = Path.cwd()

def create_tables(year):

    # load all datasets of given year
    atp_matches_df = pd.read_csv(BASE_DIR / "atp_dataset" / f"atp_matches_{year}.csv")
    atp_qual_chall_df = pd.read_csv(BASE_DIR / "atp_dataset" / f"atp_matches_qual_chall_{year}.csv")
    atp_futures_df = pd.read_csv(BASE_DIR / "atp_dataset" / f"atp_matches_futures_{year}.csv")

    # Add the tournament type to all rows
    atp_matches_df["tourney_type"] = "matches"
    atp_qual_chall_df["tourney_type"] = "challengers"
    atp_futures_df["tourney_type"] = "futures"

    source_df = pd.concat([atp_matches_df,atp_qual_chall_df,atp_futures_df])

    # Rename round to match_round to avoid calling round method 
    source_df = source_df.rename(columns={"round":"tourney_round"})

    big_table_list = []

    # counter for debugging/testing
    row_count = 0
    for x in source_df.itertuples():
        
        if not pd.isna(x.score):
            # split the rounds 
            split_rounds = x.score.split(" ")

            if len(split_rounds) == 1:
                pass
                rounds_list = (split_rounds[0],np.nan,np.nan,np.nan,np.nan)
            elif len(split_rounds) == 2:
                rounds_list = (split_rounds[0],split_rounds[1],np.nan,np.nan,np.nan)
            elif len(split_rounds) == 3:
                rounds_list = (split_rounds[0],split_rounds[1],split_rounds[2],np.nan,np.nan)
            elif len(split_rounds) == 4:
                rounds_list = (split_rounds[0],split_rounds[1],split_rounds[2],split_rounds[3],np.nan)
            elif len(split_rounds) == 5:
                rounds_list = (split_rounds[0],split_rounds[1],split_rounds[2],split_rounds[3],split_rounds[4])
        else:
            rounds_list = (np.nan,)*5

        # construct the row for winners and losers. This will create a big singular table with no normalisation.
        full_winner_row = (
            x.tourney_id,
            x.tourney_name,
            x.tourney_type,
            x.surface,
            x.draw_size,
            x.tourney_level,
            x.tourney_date,
            x.match_num,
            x.score,
            *rounds_list,
            x.best_of,
            x.tourney_round,
            x.minutes,
            x.winner_id,
            x.winner_seed,
            x.winner_name,
            x.winner_hand,
            x.winner_ht,
            x.winner_ioc,
            x.winner_age,
            x.winner_rank,
            x.winner_rank_points,
            x.w_ace,x.w_df,
            x.w_svpt,x.w_1stIn,
            x.w_1stWon,x.w_2ndWon,
            x.w_SvGms,x.w_bpSaved,
            x.w_bpFaced,
            1, # Winner is 1
            str(x.winner_id) + ":" + str(x.tourney_id) + ":" + str(x.match_num)
        )
        full_loser_row = (
            x.tourney_id,
            x.tourney_name,
            x.tourney_type,
            x.surface,
            x.draw_size,
            x.tourney_level,
            x.tourney_date,
            x.match_num,
            x.score,
            *rounds_list,
            x.best_of,
            x.tourney_round,
            x.minutes,
            x.loser_id,
            x.loser_seed,
            x.loser_name,
            x.loser_hand,
            x.loser_ht,
            x.loser_ioc,
            x.loser_age,
            x.loser_rank,
            x.loser_rank_points,
            x.w_ace,x.w_df,
            x.w_svpt,x.w_1stIn,
            x.w_1stWon,x.w_2ndWon,
            x.w_SvGms,x.w_bpSaved,
            x.w_bpFaced,
            0, # loser is 0
            str(x.loser_id) + ":" + str(x.tourney_id) + ":" + str(x.match_num)
        )

        big_table_list.append(full_winner_row)
        big_table_list.append(full_loser_row)

        # row_count += 1
        # if row_count == 5:
        #     break

    # Create a flattened table for easy analysis.
    big_table_df = pd.DataFrame(big_table_list,columns=[
                                        # Tournament data
                                        "tourney_id","tourney_name","tourney_type","surface","draw_size","tourney_level","tourney_date","match_num","score",
                                        "round_1","round_2","round_3","round_4","round_5",
                                        "best_of","tourney_round","minutes",
                                        # Player data
                                        "player_id","player_seed","player_name","player_hand","player_height","player_country","player_age","player_rank","player_rank_points",
                                        "ace","double_faults","points_on_serve","first_serve_in","1st_won","2nd_won","service_games","break_points_saved","break_points_faced",
                                        "match_outcome","player_tourney_match_id"
                                                        ])

    # Post processing to round the table
    big_table_df["player_age"] = round(big_table_df["player_age"],0)

    # create folder if it does not exist
    os.makedirs("atp_transformed", exist_ok=True)

    #big_table_df.to_csv(Path(__file__).resolve().parent / "atp_transformed" / f"{year}.csv",index=False)
    return big_table_df

# tool to make statements for all matches of a specific time period
# list_of_dfs = []

# for y in range(2000,2025):
#     if y not in [2020,2021]:
#         print(f"df_{y} = create_tables({y})")
#         list_of_dfs.append(f"df_{y}")
# print(f"df_big = pd.concat({list_of_dfs})")

df_2000 = create_tables(2000)
df_2001 = create_tables(2001)
df_2002 = create_tables(2002)
df_2003 = create_tables(2003)
df_2004 = create_tables(2004)
df_2005 = create_tables(2005)
df_2006 = create_tables(2006)
df_2007 = create_tables(2007)
df_2008 = create_tables(2008)
df_2009 = create_tables(2009)
df_2010 = create_tables(2010)
df_2011 = create_tables(2011)
df_2012 = create_tables(2012)
df_2013 = create_tables(2013)
df_2014 = create_tables(2014)
df_2015 = create_tables(2015)
df_2016 = create_tables(2016)
df_2017 = create_tables(2017)
df_2018 = create_tables(2018)
df_2019 = create_tables(2019)
df_2020 = create_tables(2020)
df_2021 = create_tables(2021)
df_2022 = create_tables(2022)
df_2023 = create_tables(2023)
df_2024 = create_tables(2024)
df = pd.concat([df_2000, df_2001, df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019,df_2020,df_2021, df_2022, df_2023, df_2024])

df["tourney_date"] = pd.to_datetime(df["tourney_date"].astype(str), format="%Y%m%d")

In [3]:
# Display dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289980 entries, 0 to 65377
Data columns (total 37 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   tourney_id               1289980 non-null  object        
 1   tourney_name             1289980 non-null  object        
 2   tourney_type             1289980 non-null  object        
 3   surface                  1289874 non-null  object        
 4   draw_size                1289980 non-null  int64         
 5   tourney_level            1289980 non-null  object        
 6   tourney_date             1289980 non-null  datetime64[ns]
 7   match_num                1289980 non-null  int64         
 8   score                    1289880 non-null  object        
 9   round_1                  1289880 non-null  object        
 10  round_2                  1284562 non-null  object        
 11  round_3                  433102 non-null   object        
 12  round_4

In [4]:
# Check for missing values
df.isnull().sum()

tourney_id                       0
tourney_name                     0
tourney_type                     0
surface                        106
draw_size                        0
tourney_level                    0
tourney_date                     0
match_num                        0
score                          100
round_1                        100
round_2                       5418
round_3                     856878
round_4                    1264980
round_5                    1283148
best_of                          0
tourney_round                    0
minutes                     932434
player_id                        0
player_seed                 848643
player_name                      0
player_hand                     15
player_height               461641
player_country                   9
player_age                    4637
player_rank                 106382
player_rank_points          106539
ace                         920970
double_faults               920970
points_on_serve     

## surface

In [5]:
value_counts = df['surface'].value_counts(dropna=False)

# Different types of surfaces
print("Value counts for 'surface' column:")
print(value_counts)

Value counts for 'surface' column:
surface
Clay      618004
Hard      601892
Carpet     35608
Grass      34370
NaN          106
Name: count, dtype: int64


### Dropping the 106 records with null surface 
It's a very low percentage of the data and will be dropped as surface cannot be guessed.

In [6]:
df = df.loc[~df['surface'].isnull()].copy()   # keeps rows where player_age is NOT null

# --- Verify removal ---
null_count = df['surface'].isnull().sum()

print("Number of rows with missing surface AFTER drop:", null_count)

Number of rows with missing surface AFTER drop: 0


## score

In [7]:
missing_score_count = df.loc[df['score'].isnull()].shape[0]

print("Missing score:", missing_score_count)

Missing score: 100


### Dropping the 100 records with null score 
It's a very low percentage of the data, score is necessary to know the outcome of a match will be dropped as surface cannot be guessed.

In [8]:
df = df.loc[~df['score'].isnull()].copy()   # keeps rows where score is NOT null

# --- Verify removal ---
null_count = df['score'].isnull().sum()

print("Number of rows with missing score AFTER drop:", null_count)

Number of rows with missing score AFTER drop: 0


## round_1

In [9]:
null_round1_count = df[df['round_1'].isna()].shape[0]

print("Amount of records with null round_1:", null_round1_count)

Amount of records with null round_1: 0


### Dropping the 70 records with null round_1 
It's a very low percentage of the data, round_1 is necessary to be sure a match was actually played.

In [10]:
df = df.loc[~df['round_1'].isnull()].copy()   # keeps rows where round_1 is NOT null

# --- Verify removal ---
null_count = df['round_1'].isnull().sum()

print("Number of rows with missing round_1 AFTER drop:", null_count)

Number of rows with missing round_1 AFTER drop: 0


### round_2 round_3 round_4 round_5

In [11]:
cols = ['score','round_1','round_2','round_3','round_4','round_5']
missing = df[cols].isna().sum()
print(missing)


score            0
round_1          0
round_2       5318
round_3     856704
round_4    1264774
round_5    1282942
dtype: int64


### Filling the remaining round records with NaN values

A lot of these rounds should be NaN because not every match consists of 5 rounds of play, these will be filled with 'N/A'.

In [12]:
round_cols = ['round_2', 'round_3', 'round_4', 'round_5']
df[round_cols] = df[round_cols].fillna('N/A')

# --- Verify removal ---
missing = df[cols].isna().sum()
print(missing)

score      0
round_1    0
round_2    0
round_3    0
round_4    0
round_5    0
dtype: int64


## player_age

In [13]:
missing_age_ids_count = df.loc[df['player_age'].isnull()].shape[0]
missing_rank_ids_count = df.loc[df['player_rank'].isnull()].shape[0]
missing_age_and_rank_ids = df.loc[df['player_age'].isnull() & df['player_rank'].isnull()].shape[0]

print("Missing player_age:", missing_age_ids_count)
print("Missing player_rank:", missing_rank_ids_count)
print("Missing both player_age and player_rank:", missing_age_and_rank_ids)

Missing player_age: 4636
Missing player_rank: 106354
Missing both player_age and player_rank: 2961


### Dropping the 2962 records with null player_ages and player_rank

These records miss valuable information and will be dropped.

In [14]:
# Count missing before dropping
missing_age_and_rank_ids = df.loc[
    df['player_age'].isnull() & df['player_rank'].isnull()
].shape[0]

print("Missing both player_age and player_rank BEFORE drop:", missing_age_and_rank_ids)

# --- Drop rows where BOTH player_age and player_rank are null ---
df = df.dropna(subset=['player_age', 'player_rank'], how='all') 

# Check again after dropping
missing_age_and_rank_ids_after = df.loc[
    df['player_age'].isnull() & df['player_rank'].isnull()
].shape[0]

print("Missing both player_age and player_rank AFTER drop:", missing_age_and_rank_ids_after)




Missing both player_age and player_rank BEFORE drop: 2961
Missing both player_age and player_rank AFTER drop: 0


In [15]:
missing_age_ids_count = df.loc[df['player_age'].isnull()].shape[0]

print("Missing player_age:", missing_age_ids_count)

# Columns to keep
cols = [
    "player_id","player_seed","player_name","player_hand","player_height","player_country",
    "player_age","player_rank","player_rank_points","ace","double_faults","points_on_serve",
    "first_serve_in","1st_won","2nd_won","service_games","break_points_saved",
    "break_points_faced","match_outcome","player_tourney_match_id"
]

# Filter rows where player_age is missing
missing_age_rows = df.loc[df['player_age'].isnull(), cols]

# Checking other columns
missing_age_rows.isnull().sum()


Missing player_age: 1675


player_id                     0
player_seed                1569
player_name                   0
player_hand                   0
player_height              1674
player_country                0
player_age                 1675
player_rank                   0
player_rank_points            0
ace                        1661
double_faults              1661
points_on_serve            1661
first_serve_in             1661
1st_won                    1661
2nd_won                    1661
service_games              1661
break_points_saved         1661
break_points_faced         1661
match_outcome                 0
player_tourney_match_id       0
dtype: int64

### Dropping the remaining 1675 records with null player_ages

Even though these records do contain_player rank, they miss a lot of valuable columns so we are going to drop them.

In [16]:
df = df.loc[~df['player_age'].isnull()].copy()   # keeps rows where player_age is NOT null

# --- Verify removal ---
missing_age_rows_after = df.loc[df['player_age'].isnull()].shape[0]

print("Number of rows with missing player_age AFTER drop:", missing_age_rows_after)

Number of rows with missing player_age AFTER drop: 0


### player_rank

In [ ]:
missing_rank_ids_count = df.loc[df['player_rank'].isnull()].shape[0]

print("Missing player_rank:", missing_rank_ids_count)

### Dropping remaining null player_rank records
We need these records removed in order to develop a good ELO rating system.

In [18]:
df = df.loc[~df['player_rank'].isnull()].copy()   # keeps rows where player_rank is NOT null

# --- Verify removal ---
missing_rank_rows_after = df.loc[df['player_rank'].isnull()].shape[0]

print("Number of rows with missing player_rank AFTER drop:", missing_age_rows_after)

Number of rows with missing player_rank AFTER drop: 0


### Player hand

In [19]:
df['player_hand'].value_counts(dropna=False)

player_hand
R    918878
L    139293
U    123089
A       485
Name: count, dtype: int64

### Filling the NaN player_hand records with U

What we don't know we fill with 'U' for unknown.

In [20]:
df['player_hand'] = df['player_hand'].replace({'L': 1, 'R': 2})

# Create missing flag
df['player_hand_missing'] = df['player_hand'].isna().astype(int)

#  Fill missing values with 'U' (Unknown)
df['player_hand'] = df['player_hand'].fillna('U')

# Normalize all codes to readable text
df['player_hand'] = df['player_hand'].replace({
    'R': 'Right',
    'L': 'Left',
    'U': 'Unknown',
    'A': 'Ambidextrous'
})

In [21]:
# open in data wrangler
df

# Save cleaned dataframe to new CSV (comment out to have it work)
output_path = Path.cwd() / "atp_transformed" / "2000-2024_clean.csv"
df.to_csv("atp_transformed/2000-2024_clean.csv", index=False)

In [22]:
# Check for missing values
df.isnull().sum()

tourney_id                      0
tourney_name                    0
tourney_type                    0
surface                         0
draw_size                       0
tourney_level                   0
tourney_date                    0
match_num                       0
score                           0
round_1                         0
round_2                         0
round_3                         0
round_4                         0
round_5                         0
best_of                         0
tourney_round                   0
minutes                    829872
player_id                       0
player_seed                743549
player_name                     0
player_hand                     0
player_height              379991
player_country                  0
player_age                      0
player_rank                     0
player_rank_points            157
ace                        818567
double_faults              818567
points_on_serve            818567
first_serve_in